In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.io as pio
import plotly.graph_objs as go
pio.renderers.default = 'notebook'


df = pd.read_csv('NQ_OHLC_1m.csv',index_col=0,parse_dates=True).tail(500000)

df['ADX'] = ta.adx(close=df['Close'],high=df['High'],low=df['Low'],length=5)['ADX_5'].shift(1)
df['ATR'] = ta.atr(close=df['Close'],high=df['High'],low=df['Low'],length=9).shift(1)
df['sma'] = ta.sma(close=df['Close'],length=9).shift(14)
df['PivotL'] = df['Low'].rolling(15).min()
df['PivotH'] = df['High'].rolling(15).max()

df['ADX_diff'] = df['ADX'].diff().rolling(3).mean() #noise reduced slope

pnl = pd.DataFrame(columns=['Entry','Exit','Bias'])

intrade = False

for j, i in df.iterrows():
    if not intrade:
        if i['ADX_diff'] < 0: #POC
            if i['High'] > i['sma'] + i['ATR']*1.5: #Short Condition
                sl = i['PivotH'] #Stop Loss
                tp = i['PivotL'] #Target
                entry = i.loc['Close']
                bias = 'short'
                intrade = True
            elif i['Low'] < i['sma'] - i['ATR']*1.5:
                sl = i['PivotL']
                tp = i['PivotH']
                entry = i.loc['Close']
                bias = 'long'
                intrade = True
    elif intrade:
        if bias == 'short':
            if i['High'] >= sl: #Stop Loss Hit
                intrade = False
                pnl.loc[j,'Entry'] = entry
                pnl.loc[j,'Exit'] = sl
                pnl.loc[j,'Bias'] = bias
            elif i['Low'] <= tp: #Take Profit Hit
                intrade = False
                pnl.loc[j,'Entry'] = entry
                pnl.loc[j,'Exit'] = tp
                pnl.loc[j,'Bias'] = bias
        elif bias == 'long':
            if i['Low'] <= sl: #Stop Loss Hit
                intrade = False
                pnl.loc[j,'Entry'] = entry
                pnl.loc[j,'Exit'] = sl
                pnl.loc[j,'Bias'] = bias
            elif i['High'] >= tp: #Take Profit Hit
                intrade = False
                pnl.loc[j,'Entry'] = entry
                pnl.loc[j,'Exit'] = tp
                pnl.loc[j,'Bias'] = bias
                

pnl['PnL'] = np.where(
    pnl['Bias'] == 'short',
    (pnl['Entry'] - pnl['Exit']) / pnl['Entry'] * 100,
    (pnl['Exit'] - pnl['Entry']) / pnl['Entry'] * 100
)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=pnl.index,
        y=pnl['PnL'].cumsum(),
        mode='lines',
        fill='tozeroy',
        line=dict(color='rgb(80,115,200)'),
        fillcolor='rgba(85,120,190,0.12)',
        hovertemplate='<b>%{x}</b><br>%{y:.2f}%',
        name='Return'
    )
)
fig.update_layout(
    template='plotly_dark',
    font=dict(family='Montserrat',size=12),
    yaxis_title='Return (%)',
    xaxis_title='Time',
    title=dict(text='Backtest Evaluation',font=dict(size=18,weight='bold'),x=0.5)
)
fig.show()

pnl.tail(25)